In [1]:
# Web scraper for KanView website that provides payroll data for the state of Kansas
# Selenium is used to handle the javascript on a website

#Importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os     #to manipulate files

In [2]:
#launch the url
url = "http://kanview.ks.gov/PayRates/PayRates_Agency.aspx"

In [3]:
chrome_path = r'/Users/jasleenarora/Desktop/setups/chromedriver'
driver = webdriver.Chrome(chrome_path)
driver.implicitly_wait(30)
driver.get(url)

In [4]:
python_button = driver.find_element_by_id("MainContent_uxLevel1_Agencies_uxAgencyBtn_33")
python_button.click()

In [5]:
# Beautiful soup is used to traverse the DOM and scrape the data

#Selenium now hands the page source to beautiful soup
soup_level1=BeautifulSoup(driver.page_source, 'lxml')

In [6]:
datalist = [] #empty list
x = 0 #counter
# Beautiful Soup will grab all the links for job titles
for link in soup_level1.find_all('a', id=re.compile("^MainContent_uxLevel2_JobTitles_uxJobTitleBtn_")):
    #Selenium visits each Job Title page
    python_button = driver.find_element_by_id('MainContent_uxLevel2_JobTitles_uxJobTitleBtn_' + str(x))
    python_button.click() #click link
    
    #Selenium hands of the source of the specific job page to Beautiful Soup
    soup_level2=BeautifulSoup(driver.page_source, 'lxml')

    #Beautiful Soup grabs the HTML table on the page
    table = soup_level2.find_all('table')[0]
    
    #Giving the HTML table to pandas to put in a dataframe object
    df = pd.read_html(str(table),header=0)
    
    #Store the dataframe in a list
    datalist.append(df[0])
    
    #Ask Selenium to click the back button
    driver.execute_script("window.history.go(-1)") 
    
    #increment the counter variable before starting the loop over
    x += 1

WebDriverException: Message: unknown error: Element <a id="MainContent_uxLevel2_JobTitles_uxJobTitleBtn_23" href="javascript:__doPostBack('ctl00$MainContent$uxLevel2_JobTitles$ctl25$uxJobTitleBtn','')">...</a> is not clickable at point (102, 710). Other element would receive the click: <html xmlns="http://www.w3.org/1999/xhtml" lang="en-us" xml:lang="en-us">...</html>
  (Session info: chrome=69.0.3497.92)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.6 x86_64)


In [ ]:
#end the Selenium browser session
driver.quit()

In [ ]:
#combine all pandas dataframes in the list into one big dataframe
result = pd.concat([pd.DataFrame(datalist[i]) for i in range(len(datalist))],ignore_index=True)

In [ ]:
#convert the pandas dataframe to JSON
json_records = result.to_json(orient='records')

In [ ]:
#get current working directory, python creates the json file
path = os.getcwd()

In [ ]:
#open, write, and close the file
f = open(path + "\\fhsu_payroll_data.json","w") #FHSU
f.write(json_records)
f.close()